In [1]:
import cv2
import numpy as np
from PIL import Image
import mediapipe as mp
import math

In [2]:
# Create a dictionary with these
body_parts={
    0:"nose",
    1:"left_eye_inner",
    2:"left_eye",
    3:"left_eve_outer",
    4:"right_eye_inner",
    5:"right_eye",
    6:"right_eye_outer",
    7:"left_ear",
    8:"right_ear",
    9:"mouth_left",
    10:"mouth_right",
    11:"left_shoulder",
    12:"right_shoulder",
    13:"left_elbow",
    14:"right_elbow",
    15:"left_wrist",
    16:"right_wrist",
    17:"left_pinky",
    18:"right_pinky",
    19:"left index",
    20:"right_index",
    21:"left_thumb",
    22:"right_thumb",
    23:"left_hip",
    24:"right_hip",
    25:"left_knee",
    26:"right_knee",
    27:"left_ankle",
    28:"right_ankle",
    29:"left_heel",
    30:"right_heel",
    31:"left_foot_index",
    32:"right_foot_index"
    }

# ref_body_dict=[]
# for i in body_parts:
#     x=body_parts[i]
#     y=i
#     ref_body_dict.append((x,y))
# ref_body_dict=dict(ref_body_dict)
# ref_body_dict.keys()

In [3]:
dimImg_points={'tree':{'right_knee':(55,426),'right_wrist':(162,88),'right_ankle':(158,409),
                        'left_ankle':(166,589),'left_wrist':(184,88),'left_knee':(183,485),'nose':(181,172)},
                'triangle':{'right_knee':(118,371),'right_wrist':(373,67),'right_ankle':(61,487),
                        'left_ankle':(374,494),'left_wrist':(363,483),'left_knee':(293,412),'nose':(406,231)},
                'wheel':{'right_knee':(144,134),'right_wrist':(455,268),'right_ankle':(90,267),
                        'left_ankle':(90,267),'left_wrist':(455,268),'left_knee':(144,134),'nose':(416,160),'left_hip':(254,49)},
                'downdog':{'right_knee':(110,153),'right_wrist':(416,261),'right_ankle':(62,250),
                        'left_ankle':(62,250),'left_wrist':(416,261),'left_knee':(110,153),'nose':(312,203),'left_hip':(211,59)}
              }

refpoints={'triangle':'left_ankle','tree':'left_ankle','wheel':['left_hip','left_ankle'],'downdog':'right_ankle'}

In [4]:
yog_pose="wheel"
dim_img=cv2.imread(f"Images\Dimension\dim_{yog_pose}_org.png")
dim_h=dim_img.shape[0]
dim_w=dim_img.shape[1]
print(dim_h,dim_w)

user_img=cv2.imread(rf'Images\Input\{yog_pose}.jpg')
user_h,user_w=user_img.shape[:2]
print(user_h,user_w)

293 504
481 721


In [5]:
def l_dist(a,b):
    sq1=(a[0]-b[0])**2
    sq2=(a[1]-b[1])**2
    res=int(math.sqrt(sq1 + sq2))
    return res

def med_pipe(img):
    mpPose = mp.solutions.pose
    pose = mpPose.Pose()
    mpDraw = mp.solutions.drawing_utils # For drawing keypoints
    points = mpPose.PoseLandmark # Landmarks
    # img = cv2.imread(r'images\tree.jpg')
    imageWidth, imageHeight = img.shape[:2]
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    blank = np.zeros(img.shape) # Blank image
    results = pose.process(imgRGB)
    if results.pose_landmarks:
        mpDraw.draw_landmarks(blank, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blank
        landmarks = results.pose_landmarks.landmark
    return results

def inference(img):
    pose_pts=med_pipe(img)
    comb_list=[]
    body_points=[]
    for id, lm in enumerate(pose_pts.pose_landmarks.landmark):
        h, w,c = img.shape
    # print(id, lm)
        cx, cy = int(lm.x*w), int(lm.y*h)
        comb_list.append((cx,cy))
    comb_list=dict(enumerate(comb_list))
    for i in comb_list:
        x=body_parts[i]
        y=comb_list[i]
        body_points.append((x,y))
    body_points=dict(body_points)
    print(body_points.keys())
    return body_points
    
def getimg_ratio(list_of_pairs,body_points,dimImg_points):
    list_of_ratios=[]
    for pair in list_of_pairs:
        user_dist=l_dist(body_points[pair[0]],body_points[pair[1]])
        dim_dist=l_dist(dimImg_points[yog_pose][pair[0]],dimImg_points[yog_pose][pair[1]])
        ratio=user_dist/dim_dist
        list_of_ratios.append(ratio)
    avg_ratio=np.average(list_of_ratios)
    print(avg_ratio)
    return avg_ratio

def scaling(dim_img,ratio,dimImg_points,yog_pose):
    dim_image_scaled=cv2.resize(dim_img,(int(dim_w*ratio),int(dim_h*ratio)))
    for body_part in dimImg_points[yog_pose].keys():
        dimImg_points[yog_pose][body_part]=[int(dimImg_points[yog_pose][body_part][0]*ratio),int(dimImg_points[yog_pose][body_part][1]*ratio)]
    print(dim_image_scaled.shape)
    print(dimImg_points)
    return dimImg_points,dim_image_scaled

def add_padding(user_img,dim_img_scaled,yog_pose,dimImg_points_scaled,body_points):
    leftdist=dimImg_points_scaled[yog_pose][refpoints[yog_pose]][0]
    rightdist=dim_img_scaled.shape[1]-leftdist
    topdist=dimImg_points_scaled[yog_pose][refpoints[yog_pose]][1]
    bottomdist=dim_img_scaled.shape[0]-topdist
    
    user_leftdist=body_points[refpoints[yog_pose]][0]
    user_rightdist=user_img.shape[1]-user_leftdist
    user_topdist=body_points[refpoints[yog_pose]][1]
    user_bottomdist=user_img.shape[0]-user_topdist

    right =user_rightdist-rightdist
    left = user_leftdist-leftdist
    top = user_topdist-topdist
    bottom =user_bottomdist-bottomdist
    print(right,left,top,bottom)
    result=cv2.copyMakeBorder(dim_img_scaled, top, bottom, left, right, cv2.BORDER_CONSTANT,(0,0,0))
    print(result.shape)
    return result

def transparent_overlay(user_img,dimimg_padded):
    background = user_img
    overlay=dimimg_padded
    alpha_channel = overlay[:, :, 2] / 255 # convert from 0-255 to 0.0-1.0
    overlay_colors = overlay[:, :, :3]
    alpha_mask = np.dstack((alpha_channel, alpha_channel, alpha_channel))
    # The background image is larger than the overlay so we'll take a subsection of the background that matches the
    # dimensions of the overlay.
    # NOTE: For simplicity, the overlay is applied to the top-left corner of the background(0,0). An x and y offset
    # could be used to place the overlay at any position on the background.
    h, w = overlay.shape[:2]
    background_subsection = background[0:h, 0:w]
    # combine the background with the overlay image weighted by alpha
    composite = background_subsection * (1 - alpha_mask) + overlay_colors * alpha_mask
    # overwrite the section of the background image that has been updated
    background[0:h, 0:w] = composite
    cv2.imwrite(f'Images\Overlay\{yog_pose}_overlay.png', background)
    cv2.imshow('combined.png', background)
    cv2.waitKey()
    cv2.destroyAllWindows()


In [6]:
def add_padding(user_img,dim_img_scaled,yog_pose,dimImg_points_scaled,body_points):
    if yog_pose=='wheel':
        leftdist=dimImg_points_scaled[yog_pose][refpoints[yog_pose][0]][0]
        rightdist=dim_img_scaled.shape[1]-leftdist
        topdist=dimImg_points_scaled[yog_pose][refpoints[yog_pose][1]][1]
        bottomdist=dim_img_scaled.shape[0]-topdist
        
        user_leftdist=body_points[refpoints[yog_pose][0]][0]
        user_rightdist=user_img.shape[1]-user_leftdist
        user_topdist=body_points[refpoints[yog_pose][1]][1]
        user_bottomdist=user_img.shape[0]-user_topdist
    else:
        leftdist=dimImg_points_scaled[yog_pose][refpoints[yog_pose]][0]
        rightdist=dim_img_scaled.shape[1]-leftdist
        topdist=dimImg_points_scaled[yog_pose][refpoints[yog_pose]][1]
        bottomdist=dim_img_scaled.shape[0]-topdist
        
        user_leftdist=body_points[refpoints[yog_pose]][0]
        user_rightdist=user_img.shape[1]-user_leftdist
        user_topdist=body_points[refpoints[yog_pose]][1]
        user_bottomdist=user_img.shape[0]-user_topdist

    right =user_rightdist-rightdist
    left = user_leftdist-leftdist
    top = user_topdist-topdist
    bottom =user_bottomdist-bottomdist
    print(right,left,top,bottom)
    result=cv2.copyMakeBorder(dim_img_scaled, top, bottom, left, right, cv2.BORDER_CONSTANT,(0,0,0))
    print(result.shape)
    return result

In [7]:
body_points=inference(user_img)

dict_keys(['nose', 'left_eye_inner', 'left_eye', 'left_eve_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index'])


In [8]:
ratio=getimg_ratio([['left_knee','left_ankle'],['right_ankle','right_knee']],body_points,dimImg_points)

1.174825174825175


In [9]:
dimImg_points_scaled,dim_img_scaled=scaling(dim_img,ratio,dimImg_points,yog_pose)

(344, 592, 3)
{'tree': {'right_knee': (55, 426), 'right_wrist': (162, 88), 'right_ankle': (158, 409), 'left_ankle': (166, 589), 'left_wrist': (184, 88), 'left_knee': (183, 485), 'nose': (181, 172)}, 'triangle': {'right_knee': (118, 371), 'right_wrist': (373, 67), 'right_ankle': (61, 487), 'left_ankle': (374, 494), 'left_wrist': (363, 483), 'left_knee': (293, 412), 'nose': (406, 231)}, 'wheel': {'right_knee': [169, 157], 'right_wrist': [534, 314], 'right_ankle': [105, 313], 'left_ankle': [105, 313], 'left_wrist': [534, 314], 'left_knee': [169, 157], 'nose': [488, 187], 'left_hip': [298, 57]}, 'downdog': {'right_knee': (110, 153), 'right_wrist': (416, 261), 'right_ankle': (62, 250), 'left_ankle': (62, 250), 'left_wrist': (416, 261), 'left_knee': (110, 153), 'nose': (312, 203), 'left_hip': (211, 59)}}


In [10]:
dimimg_padded=add_padding(user_img,dim_img_scaled,yog_pose,dimImg_points_scaled,body_points)

92 37 71 66
(481, 721, 3)


In [11]:
# cv2.imshow("user",user_img)
# cv2.imshow("dim",dimimg_padded)
print(user_img.shape)
print(dimimg_padded.shape)
# cv2.imwrite(f"Images\Dimension padded\dim_{yog_pose}_padded.png",dimimg_padded)
# cv2.waitKey()
# cv2.destroyAllWindows()

(481, 721, 3)
(481, 721, 3)


In [12]:
# dim_transparent=cv2.imread(f"Images\Dimension transparent\dimimgpadded_{yog_pose}_transparent.png",cv2.IMREAD_UNCHANGED)
transparent_overlay(user_img,dimimg_padded)

In [13]:
# dimimg_padded = Image.open(f"Images\Dimension padded\dim_{yog_pose}_padded.png")
# def convertImage(dimimg_padded):
#     # img = Image.open(f"Images_output\dimimg_padded.png")
#     img = dimimg_padded.convert("RGBA")
  
#     datas = img.getdata()
  
#     newData = []
  
#     for item in datas:
#         if (item[0] == 255 and item[1] == 255 and item[2] == 255) or (item[0] == 0 and item[1] == 0 and item[2] == 0):
#             newData.append((255, 255, 255, 0))
#         else:
#             newData.append(item)
  
#     img.putdata(newData)
#     img.save(f"Images\Dimension transparent\dimimgpadded_{yog_pose}_transparent.png", "PNG")
#     print("Successful")
#     # return img 

# convertImage(dimimg_padded)